### Load Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import joblib
corpus = joblib.load("/content/drive/MyDrive/corpus.joblib")
print(len(corpus))

4568371


### Functions

In [118]:
!wget https://raw.githubusercontent.com/aljubaer/Topic-Modeling-in-Bengali/master/code/stop_words.txt

--2024-05-19 13:55:15--  https://raw.githubusercontent.com/aljubaer/Topic-Modeling-in-Bengali/master/code/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5443 (5.3K) [text/plain]
Saving to: ‘stop_words.txt’

stop_words.txt      100%[===================>]   5.32K  --.-KB/s    in 0s      

2024-05-19 13:55:15 (94.0 MB/s) - ‘stop_words.txt’ saved [5443/5443]



In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
import json
import os


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
!pip install sbnltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusol

In [126]:
stopwords_file = open('stop_words.txt', "r+", encoding = 'utf-8')
all_stopwords = stopwords_file.read()
stopwords_ready = [word.strip() for word in all_stopwords.split()]
print(stopwords_ready)

def convertToDataFrame(data_json):
    df = pd.DataFrame(data_json)
    print('DataFrame shape' + str(df.shape))
    return df

def remove_stopwords(content):
    without_stopwords = []
    for word in content:
        if word not in stopwords_ready and len(word) > 2:
            without_stopwords.append(word)
    return without_stopwords
def remove_stopwords_list(data_list):
    data_without_stopwords_list = []
    for content in data_list:
        data_without_stopwords_list.append(remove_stopwords(content))
    return data_without_stopwords_list
    def valid_bengali_letters(char):
      return ord(char) >= 2433 and ord(char) <= 2543

def get_replacement(char):
    if valid_bengali_letters(char):
        return char
    newlines = [10, 2404, 2405, 2551, 9576]
    if ord(char) in newlines:
        return ' '
    return ' ';

def get_valid_lines(line):
    copy_line = ''
    for letter in line:
        copy_line += get_replacement(letter)
    return copy_line

def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(get_valid_lines(sentence)))  # deacc=True removes punctuations

from sbnltk.Stemmer import stemmerOP
bn_stemmer=stemmerOP()
def stemming_data(content):
    for i, word in enumerate(content):
        content[i] = bn_stemmer.stemWord(word)
    return content
def stemming_data_list(data_list):
    for i, sdata in enumerate(data_list):
        data_list[i] = stemming_data(sdata)
    return data_list
def valid_bengali_letters(char):
    return ord(char) >= 2433 and ord(char) <= 2543


['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [128]:
def preprocess_data(data_json):
    data_tokenized_list = list(sent_to_words(data_json))
    print(data_tokenized_list)
    data_without_stopwords_list = remove_stopwords_list(data_tokenized_list)
    print(data_without_stopwords_list)
    data_stemmed_list = stemming_data_list(data_without_stopwords_list)
    return data_stemmed_list

In [130]:
preprocess_data(["<p>আমার সোনার বাংলা </p>", "<p>আমার সোনার বাংলা </p>"])

[['আমার', 'সোনার', 'বাংলা'], ['আমার', 'সোনার', 'বাংলা']]
[['সোনার', 'বাংলা'], ['সোনার', 'বাংলা']]


[['সোন', 'বাংলা'], ['সোন', 'বাংলা']]

### Batch Processing

In [131]:
len(corpus)

4568371

In [132]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm

# Define the Word2Vec model using PyTorch
class Word2VecPytorch(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecPytorch, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_words):
        embeds = self.embeddings(input_words)
        output = self.linear(embeds)
        return output

def save_checkpoint(model, epoch, optimizer, loss):
    model_dir = "checkpoints"
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, f"word2vec_model_epoch{epoch}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, model_path)
    print(f"Checkpoint saved: {model_path}")

In [ ]:
batch_size = 100000
num_batches = 4
data_batches = []

for i in tqdm(range(num_batches), desc="Loading Batches", unit="batch"):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size
    data_batch = corpus[start_index:end_index]
    preprocessed_batch = preprocess_data(data_batch)
    data_batches.append(preprocessed_batch)

Loading Batches:   0%|          | 0/4 [00:00<?, ?batch/s]

In [40]:
processed_sentences = []
for batch in tqdm(data_batches, desc="Flattening Batches", unit="batch"):
    for sentence in batch:
        processed_sentences.append(sentence)

Flattening Batches: 100%|██████████| 4/4 [00:00<00:00, 102.95batch/s]


In [57]:
from collections import Counter
words = []
for sentence in tqdm(processed_sentences, desc=f"Counting ", unit="sentence"):
  for word in sentence:
    words.append(word)
word_counter = Counter(words)
vocab = {word: idx for idx, (word, _) in enumerate(word_counter.items())}
vocab_size = len(vocab)
print(vocab_size)

Counting : 100%|██████████| 400000/400000 [00:03<00:00, 113831.48sentence/s]


1194943


In [58]:
embedding_dim = 400

# Initialize Word2Vec model with PyTorch
model = Word2VecPytorch(vocab_size, embedding_dim)
model = model.cuda()  # Move model to GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [49]:
tempv = list(vocab.keys())
len(tempv)

189179

In [52]:
tempv[100:130]

['পরে',
 'করোনা',
 'ধরে',
 'তাই',
 'তখন',
 'বিষয়ে',
 'ভালো',
 'বছরের',
 'মো.',
 'কী',
 'আবার',
 'তৈরি',
 '১',
 'মানুষের',
 'কোটি',
 'জন',
 'তথ্য',
 'সেখানে',
 'তিন',
 'না,',
 'সালের',
 'তারা',
 'বাংলাদেশের',
 'দেশে',
 'কি',
 'পাওয়া',
 'যায়।',
 'হয়,',
 'করে।',
 'নেই।']

In [54]:
# Training loop with progress bar
for epoch, data_batch in enumerate(data_batches):
    total_loss = 0
    for sentence in tqdm(data_batch, desc=f"Epoch {epoch + 1}/{num_batches}", unit="sentence"):
        for target_word in sentence:
            try:
              context_words = [word for word in sentence if word != target_word]
              target_idx = torch.tensor([vocab[target_word]], dtype=torch.long).cuda()
              context_idxs = torch.tensor([vocab[word] for word in context_words], dtype=torch.long).cuda()

              optimizer.zero_grad()
              output = model(context_idxs)
              loss = criterion(output, target_idx)
              loss.backward()
              optimizer.step()
              total_loss += loss.item()
              print(f"{target_word} found!")

            except:
              #print(f"{target_word} not found!")
              pass

    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")
    save_checkpoint(model, epoch, optimizer, total_loss)

Epoch 1/4:   0%|          | 87/100000 [00:08<2:34:04, 10.81sentence/s]


KeyboardInterrupt: 

In [ ]:

# Save the final trained model
final_model_path = "final_word2vec_model.pt"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved: {final_model_path}")
